<a href="https://colab.research.google.com/github/zacharylazzara/tent-detection/blob/main/Tent_Detector_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description
This notebook detects tents in satellite images using UNet.

#Settings

In [1]:
#@title Initial Imports
import os
from google.colab import drive
from google.colab import files
from google.colab import output
from google.colab import runtime

In [2]:
#@title Models

DOWNLOAD_MODEL            = False #@param {type:"boolean"}
UPLOAD_MODEL              = False #@param {type:"boolean"}
FORCE_UPLOAD              = False #@param {type:"boolean"}
SEGMENTOR_MODEL_FILENAME  = 'unet.pth' #@param {type:"string"}
DOWNLOAD_MODEL_URL        = 'https://github.com/zacharylazzara/tent-detection/blob/685f80327f493cf816fceed7a2094654f3107bc2/unet.pth' #@param {type:"string"}

uploaded_files = None
if FORCE_UPLOAD and (UPLOAD_MODEL or DOWNLOAD_MODEL):
  os.remove(SEGMENTOR_MODEL_FILENAME)
if DOWNLOAD_MODEL:
  !wget $DOWNLOAD_MODEL_URL
  if not (os.path.exists(SEGMENTOR_MODEL_FILENAME)):
    raise Exception(f'File "{SEGMENTOR_MODEL_FILENAME}" not found!')
else:
  if UPLOAD_MODEL and not (os.path.exists(SEGMENTOR_MODEL_FILENAME)):
    uploaded_files = files.upload()
    for filename in uploaded_files.keys():
      print(f'Uploaded file "{filename}"')
      if filename != SEGMENTOR_MODEL_FILENAME:
        raise Exception('Filename must match SEGMENTOR_MODEL_FILENAME!')
  if UPLOAD_MODEL and uploaded_files == {}:
    raise Exception('No files uploaded!')

Saving unet.pth to unet.pth
Uploaded file "unet.pth"


In [3]:
#@title Output
SAVE_TO_GOOGLE_DRIVE      = True #@param {type:"boolean"}
DOWNLOAD_OUTPUT           = False #@param {type:"boolean"}
PLAY_SOUND_ON_COMPLETE    = True #@param {type:"boolean"}
KILL_RUNTIME_ON_COMPLETE  = True #@param {type:"boolean"}
MINUTES_TO_KILL_RUNTIME   = 5 #@param {type:"slider", min:0, max:30, step:1}
MINUTES_TO_KILL_RUNTIME   = MINUTES_TO_KILL_RUNTIME*60

In [4]:
#@title Config
TRAIN_MODEL   = True #@param {type:"boolean"}
N_EPOCHS      = 200 #@param {type:"number"}
BATCH_SIZE    = 8 #@param {type:"number"}
INIT_LR       = 0.0001 #@param {type:"number"}
IMAGE_HEIGHT  = 512 #@param {type:"number"}
IMAGE_WIDTH   = 512 #@param {type:"number"}
TEST_SPLIT    = 0.15 #@param {type:"number"}
RANDOM_STATE  = 42 #@param {type:"number"}
OUTPUT_FORMAT = 'png' #@param ["png", "jpg"] {allow-input: true}
INPUT_FORMAT  = 'jpg' #@param ["png", "jpg"] {allow-input: true}

#Definitions

##Initialization

In [5]:
#@title Imports
!pip install -q tqdm-thread
!pip install torchmetrics -q
import time
import math
import csv
import cv2
import torch
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision.transforms.functional as TF
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from statistics import mean
from tqdm.auto import tqdm
from tqdm_thread import tqdm_thread
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch.nn import Sequential
from torch.nn import ModuleList
from torch.nn import ConvTranspose2d
from torch.nn import Flatten
from torch.nn import functional
from torch.nn import BatchNorm2d
from torch.nn import Softplus
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.nn.modules.loss import PoissonNLLLoss
from torchmetrics.classification import BinaryJaccardIndex
from torch import flatten
from torch import cat
from torch import randn
from torchvision import transforms
from torchvision.transforms import CenterCrop
from torchvision.utils import save_image
from torch.utils.data import Dataset
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 5.2 MB/s eta 0:00:00


In [6]:
#@title Device
DEVICE = None
if torch.cuda.is_available():
    DEVICE = 'cuda'
elif torch.backends.mps.is_available():
    DEVICE = 'mps'
else:
    DEVICE = 'cpu'
PIN_MEMORY = True if DEVICE != 'cpu' else False
print(f'Using device: {DEVICE}')

Using device: cpu


In [7]:
#@title Paths
SRC_PATH            = os.environ['SRC_PATH']            = f'sarpol-zahab-tents'
OUTPUT_PATH         = os.environ['OUTPUT_PATH']         = f'output'

DATA_PATH           = os.environ['DATA_PATH']           = f'{SRC_PATH}/data'
IMAGES_PATH         = os.environ['IMAGES_PATH']         = f'{DATA_PATH}/images'
MASKS_PATH          = os.environ['MASKS_PATH']          = f'{DATA_PATH}/labels'
LABELS_PATH         = os.environ['LABELS_PATH']         = f'{DATA_PATH}/sarpol_counts.csv'

MODELS_PATH         = os.environ['MODELS_PATH']         = f'{OUTPUT_PATH}/models'
METRICS_PATH        = os.environ['METRICS_PATH']        = f'{OUTPUT_PATH}/metrics'
PREDS_PATH          = os.environ['PREDS_PATH']          = f'{OUTPUT_PATH}/predictions'

REGIONS_PATH        = os.environ['REGIONS_PATH']        = f'{OUTPUT_PATH}/regions'
REGIONS_Y_PATH      = os.environ['REGIONS_Y_PATH']      = f'{REGIONS_PATH}/truths'
REGIONS_P_PATH      = os.environ['REGIONS_P_PATH']      = f'{REGIONS_PATH}/predictions'

OVERVIEW_PATH       = os.environ['OVERVIEW_PATH']       = f'{OUTPUT_PATH}/overviews'
OVERLAY_PATH        = os.environ['OVERLAY_PATH']        = f'{OUTPUT_PATH}/overlays'
OVERLAY_Y_PATH      = os.environ['OVERLAY_Y_PATH']      = f'{OVERLAY_PATH}/truths'
OVERLAY_P_PATH      = os.environ['OVERLAY_P_PATH']      = f'{OVERLAY_PATH}/predictions'
OVERLAY_PR_PATH     = os.environ['OVERLAY_PR_PATH']     = f'{OVERLAY_P_PATH}/regions'
OVERLAY_YR_PATH     = os.environ['OVERLAY_YR_PATH']     = f'{OVERLAY_Y_PATH}/regions'

G_DRIVE_MOUNT_POINT = os.environ['G_DRIVE_MOUNT_POINT'] = f'g_drive'
G_DRIVE_STORAGE     = os.environ['G_DRIVE_STORAGE']     = f'{G_DRIVE_MOUNT_POINT}/MyDrive'

In [8]:
#@title Mount
if SAVE_TO_GOOGLE_DRIVE:
  !mkdir -p $G_DRIVE_MOUNT_POINT
  drive.mount(G_DRIVE_MOUNT_POINT)

Mounted at g_drive


In [9]:
#@title Environment
%%bash

if [ -d 'sample_data' ]; then
  rm -r sample_data
fi

if [ ! -d $SRC_PATH ]; then
  git clone https://github.com/tofighi/sarpol-zahab-tents.git
fi

if [ ! -d $OUTPUT_PATH ]; then
  mkdir -p $MODELS_PATH
  mkdir -p $METRICS_PATH
  mkdir -p $PREDS_PATH
  mkdir -p $REGIONS_PATH
  mkdir -p $REGIONS_Y_PATH
  mkdir -p $REGIONS_P_PATH
  mkdir -p $OVERVIEW_PATH
  mkdir -p $OVERLAY_PATH
  mkdir -p $OVERLAY_Y_PATH
  mkdir -p $OVERLAY_P_PATH
  mkdir -p $OVERLAY_PR_PATH
  mkdir -p $OVERLAY_YR_PATH
fi

Cloning into 'sarpol-zahab-tents'...


##Models

In [10]:
#@title UNet
# Adapted from: https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/
class Block(Module):
  def __init__(self, in_channels, out_channels):
    super(Block, self).__init__()
    self.double_conv2d = Sequential(
        Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        BatchNorm2d(out_channels),
        ReLU(inplace=True),
        Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        BatchNorm2d(out_channels),
        ReLU(inplace=True)
    )

  def forward(self, x):
    return self.double_conv2d(x)

class Encoder(Module):
  def __init__(self, channels=(3, 16, 32, 64)):
    super(Encoder, self).__init__()
    self.encoder_blocks = ModuleList([Block(channels[i], channels[i+1]) for i in range(len(channels)-1)])
    self.pool = MaxPool2d(2)

  def forward(self, x):
    block_outputs = []
    for block in self.encoder_blocks:
      x = block(x)
      block_outputs.append(x)
      x = self.pool(x)
    return block_outputs

class Decoder(Module):
  def __init__(self, channels=(64, 32, 16)):
    super(Decoder, self).__init__()
    self.up_convs = ModuleList([ConvTranspose2d(channels[i], channels[i+1], 2, 2) for i in range(len(channels)-1)])
    self.decoder_blocks = ModuleList([Block(channels[i], channels[i+1]) for i in range(len(channels)-1)])
  
  def crop(self, encoder_features, x):
    (_, _, H, W) = x.shape
    return CenterCrop([H, W])(encoder_features)
  
  def forward(self, x, encoder_features):
    for i in range(len(self.up_convs)):
      x = self.up_convs[i](x)
      encoder_feature = self.crop(encoder_features[i], x)
      x = cat([x, encoder_feature], dim=1)
      x = self.decoder_blocks[i](x)
    return x

class UNet(Module):
  def __str__(self) -> str:
    return 'UNet'

  def __init__(self, encoder_channels=(3, 16, 32, 64), decoder_channels=(64, 32, 16), classes=1, retain_dim=True, output_size=(512, 512)):
    super(UNet, self).__init__()
    self.encoder = Encoder(encoder_channels)
    self.decoder = Decoder(decoder_channels)
    self.head = Conv2d(decoder_channels[-1], classes, 1)
    self.retain_dim = retain_dim
    self.output_size = output_size

  def forward(self, x):
    encoder_features = self.encoder(x)
    decoder_features = self.decoder(encoder_features[::-1][0], encoder_features[::-1][1:])
    map = self.head(decoder_features)
    if self.retain_dim:
      map = functional.interpolate(map, self.output_size)
    return map

In [11]:
#@title Dataset
# Adapted from: https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/
class SegmentationDataset(Dataset):
  def __init__(self, dataframe, transformations = None):
    self.dataframe = dataframe
    self.transformations = transformations

  def __len__(self):
    return len(self.dataframe.index)

  def __getitem__(self, index):
    image = cv2.cvtColor(cv2.imread(self.dataframe.iloc[index]['image_paths']), cv2.COLOR_BGR2RGB)
    mask = cv2.threshold(cv2.imread(self.dataframe.iloc[index]['mask_paths'], cv2.IMREAD_GRAYSCALE), 150, 255, cv2.THRESH_BINARY)[1]
    if self.transformations is not None:
      image = self.transformations(image)
      mask = self.transformations(mask)
    return (image, mask, self.dataframe.iloc[index]['labels'], self.dataframe.index[index])

##Functions

In [12]:
#@title Load
def load_data(images_path=IMAGES_PATH, masks_path=MASKS_PATH, csv_path=LABELS_PATH):
  """
  Returns a dataframe with the feature and target paths, along with the image
  name and number of tents (i.e., labels). The dataset must be square otherwise
  tiling will fail.
  """
  with open(csv_path) as csv_file:
    rows = [row for row in csv.reader(csv_file)]
    assert len(rows) % 2 == 0 # We need our data to be square in order to tile it later
  return pd.DataFrame({
    'names'        : [row[0].split('.')[0] for row in rows],
    'image_paths'  : [str(next(Path(images_path).glob(row[0]))) for row in rows],
    'mask_paths'   : [str(next(Path(masks_path).glob(row[0]))) for row in rows],
    'labels'       : [int(row[1]) for row in rows]
  }).set_index('names').astype({'labels': 'int'})

In [13]:
#@title Train
# Adapted from: https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/
def train(model, t_loader, v_loader, loss_func, opt, metric=None, epochs=N_EPOCHS):
  """Trains the model and outputs its metrics in a dataframe (loss and accuracy)."""

  history = pd.DataFrame({
      't': {'losses':[], 'metrics':[]},
      'v': {'losses':[], 'metrics':[]}
  })

  if metric != None:
    metric.to(DEVICE)

  progress_bar = tqdm(range(epochs))
  for e in progress_bar:
    model.train()
    losses = []
    for (i, (x, y, _, _)) in enumerate(t_loader):
      (x, y) = (x.to(DEVICE), y.to(DEVICE))

      pred = model(x)
      loss = loss_func(pred, y)
      losses.append(loss.item())

      if loss.requires_grad:
        opt.zero_grad()
        loss.backward()
      opt.step()

      if metric != None:
        metric.update(pred, y)
        
    history['t']['losses'].append(mean(losses))
    if metric != None:
      history['t']['metrics'].append(metric.compute().cpu().detach().numpy())
      metric.reset()
    
    with torch.no_grad():
      model.eval()
      
      losses = []
      for (x, y, _, _) in v_loader:
        (x, y) = (x.to(DEVICE), y.to(DEVICE))

        pred = model(x)
        loss = loss_func(pred, y)
        losses.append(loss.item())

        if metric != None:
          metric.update(pred, y)
      
      history['v']['losses'].append(mean(losses))
      if metric != None:
        history['v']['metrics'].append(metric.compute().cpu().detach().numpy())
        metric.reset()

    progress_bar.set_description(f'Epoch({e+1}/{N_EPOCHS}) Training {model}, Train Loss: {history["t"]["losses"][-1]:.4f}, Test Loss: {history["v"]["losses"][-1]:.4f}')
  return history

In [14]:
#@title Predict
def predict(model, loader, output_dir=PREDS_PATH):
  """
  Runs the model without training and saves the predictions to the output
  directory. Returns a dataframe containing the image paths, mask paths (i.e,
  prediction paths), and the tent count as determined by counting contours in the
  prediction.
  """
  preds = []
  with torch.no_grad():
    model.eval()
    progress_bar = tqdm(loader)
    progress_bar.set_description(f'Evaluating {model}')
    for (x, y, _, name) in progress_bar:
      (x, y) = (x.to(DEVICE), y.to(DEVICE))
      p = model(x)
      for batch, img in enumerate(p.cpu().detach()):
        out_path = f'{output_dir}/{name[batch]}.{OUTPUT_FORMAT}'
        save_image(img, out_path)
        preds.append({'names':name[batch], 'image_paths':str(next(Path(IMAGES_PATH).glob(f'{name[batch]}.{INPUT_FORMAT}'))), 'mask_paths':out_path, 'labels':count_contours(img)})
        progress_bar.set_description(f'Saved prediction for {name[batch]} to {out_path}')
  return pd.DataFrame(preds).set_index('names').fillna(np.nan)

In [15]:
#@title Region
def region_box(name, c, w, h):
  """Draws the region box with the tent count and name."""
  shape = [(0, 0), (w-1, h-1)]
  
  img = Image.new('RGBA', (w, h))
  draw = ImageDraw.Draw(img)
  font = ImageFont.truetype('LiberationMono-Regular.ttf', 50)
  draw.text((10, 10), f'{c}', font=font, fill=(255, 0, 0))

  _, _, tw, th = draw.textbbox((0, 0), name, font=font)

  tw = w-tw
  th = h-th

  draw.text((tw-20, th-10), name, font=font, fill=(255, 0, 0))

  rec = ImageDraw.Draw(img)  
  rec.rectangle(shape, fill = None, outline ='red')

  return img

def region(loader, output_dir=REGIONS_PATH, overview_name='regions_overview'):
  """
  Generates an outline around each image and puts the number of tents
  in the upper left corner of the image, with the image name in the bottom right.
  Mostly redundant since we're using Seaborn for the heatmap anyway, but may
  be useful for troubleshooting.
  """
  region_paths = []
  progress_bar = tqdm(loader)
  progress_bar.set_description(f'Creating region overlays...')
  region_overview = None
  for (_, y, c, name) in progress_bar:
    regions = []
    for batch, img in enumerate(y.cpu().detach()):
      out_path = f'{output_dir}/{name[batch]}.{OUTPUT_FORMAT}'
      region = region_box(name[batch], c[batch], img.shape[1], img.shape[2])
      region.save(out_path)
      region_paths.append({'names':name[batch], 'region_paths':out_path})
      progress_bar.set_description(f'Saved region overlay for {name[batch]} to {out_path}')

      transform = transforms.Compose([transforms.PILToTensor()])
      region = transform(region)
      regions.append(region)

    region_overview = tiler(region_overview, regions).double()
  save_image(region_overview, f'{OVERVIEW_PATH}/{overview_name}.{OUTPUT_FORMAT}')
  return pd.DataFrame(region_paths).set_index('names').fillna(np.nan)

In [16]:
#@title Tile
def tiler(tiles, tile_row):
  """Takes a row of tiles. The total number of tiles must be divisible by 2."""
  if tiles == None:
    tiles = torch.cat(tuple(tile_row), 2)
  else:
    tiles = torch.cat((tiles, torch.cat(tuple(tile_row), 2)), 1)
  return tiles
 
def tile(loader, output_path_x, output_path_y):
  """Tiles using a loader. Total number of tiles must be divisible by 2."""
  output_x = output_y = None
  progress_bar = tqdm(loader)
  progress_bar.set_description('Tiling')
  for (x, y, _, _) in progress_bar:
    if output_path_x != None:
      output_x = tiler(output_x, x)
    if output_path_y != None:
      output_y = tiler(output_y, y)

  if output_x != None:
    print(f'Saving {output_path_x}...')
    save_image(output_x, output_path_x)
  if output_y != None:
    print(f'Saving {output_path_y}...')
    save_image(output_y, output_path_y)
  print('Done.')

In [17]:
#@title Count Contours
# Adapted from https://stackoverflow.com/questions/48154642/how-to-count-number-of-dots-in-an-image-using-python-and-opencv
def contours(y):
  img = y.numpy().T.astype(np.uint8).copy()

  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
  closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
  cnts = cv2.findContours(closing, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2]

  max_area = 20
  xcnts = []
  for cnt in cnts:
    if cv2.contourArea(cnt) < max_area:
      xcnts.append(cnt)

  return xcnts

def count_contours(y):
  return int(len(contours(y)))

def localize_contours(y):
  """
  Note that not all detected contours will be localizable, as some are opened
  (i.e., look like a C) and thus result in a division by zero error (thus we 
  use isContourConvex() to prevent this, skipping the opened contours).
  """
  moments = [cv2.moments(cnt, True) for cnt in contours(y) if cv2.isContourConvex(cnt)]
  coordinates = [(m['m10']/m['m00'], m['m01']/m['m00']) for m in moments]
  return coordinates

def count(loader):
  """Unused."""
  preds = []
  progress_bar = tqdm(loader)
  progress_bar.set_description('Counting Contours...')
  for (_, y, _, name) in progress_bar:
    y = y.to(DEVICE)
    for batch, img in enumerate(y.cpu().detach()):
      preds.append({'names':name[batch], 'image_paths':str(next(IMAGES_PATH.glob(f'{name[batch]}{INPUT_FORMAT}'))), 'mask_paths':None, 'labels':count_contours(img)})
  return pd.DataFrame(preds).set_index('names').fillna(np.nan)

In [18]:
#@title Overlay
def overlay(background_path, foreground_path, output_path, bg_opacity = 1, fg_opacity = 1):
  with Image.open(background_path).convert('RGBA') as background:
    background = np.array(background)
  with Image.open(foreground_path).convert('RGBA') as foreground:
    foreground = np.array(foreground)

  for channel in range(1, 2):
    foreground[foreground[:,:,channel] > 0, channel] = 0

  overlay = cv2.addWeighted(background, bg_opacity, foreground, fg_opacity, 0)
  output = Image.fromarray(overlay)
  output.save(output_path)

  return output_path

In [19]:
#@title Heatmap
def heatmap(image_path, dataframe, output_path, title):
  data = [x for x in np.array_split(dataframe['labels'].replace(0, np.nan).tolist(), int(math.sqrt(dataframe.shape[0])))]

  sns.set(font_scale=1)
  fig, ax = plt.subplots(figsize=(15, 15))
  ax.set_title(title)
  
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  ax.tick_params(left=False, bottom=False)
  sns.heatmap(data, annot=True, square=True, fmt='.5g', alpha=0.3, zorder=2, cbar_kws={'shrink': 0.7}, ax=ax)

  with Image.open(image_path).convert("RGB") as image:
    ax.imshow(image, aspect=ax.get_aspect(), extent=ax.get_xlim()+ax.get_ylim(), zorder=1)
  plt.savefig(output_path, bbox_inches='tight')
  plt.close()

#Main

##Dataset

In [20]:
#@title Training and Validation Loaders
transformations = transforms.Compose([transforms.ToPILImage(), transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()])

# Split dataset into training and validation subsets
training_data, validation_data = train_test_split(load_data(), test_size=TEST_SPLIT, random_state=RANDOM_STATE)
training_dataset = SegmentationDataset(training_data, transformations)
validation_dataset = SegmentationDataset(validation_data, transformations)

t_loader = DataLoader(training_dataset, shuffle=True, batch_size=BATCH_SIZE, pin_memory=PIN_MEMORY, num_workers=os.cpu_count(), persistent_workers=True)
v_loader = DataLoader(validation_dataset, shuffle=True, batch_size=BATCH_SIZE, pin_memory=PIN_MEMORY, num_workers=os.cpu_count(), persistent_workers=True)

In [21]:
#@title Split Ratio
t_count = len(training_data)
v_count = len(validation_data)

t_ratio = 1-v_count/t_count
v_ratio = v_count/t_count

print(f'Training to Validation Ratio\n')
print(f'Training ({t_count}): \t{t_ratio*100:>10.2f}%')
print(f'Validation ({v_count}): \t{v_ratio*100:>10.2f}%')
print(f'Total ({t_count + v_count}): \t\t{t_ratio*100 + v_ratio*100:>10.2f}%') 

assert t_ratio + v_ratio == 1 # Sanity Check

Training to Validation Ratio

Training (217): 	     82.03%
Validation (39): 	     17.97%
Total (256): 		    100.00%


## Model Operations

In [22]:
#@title Load Models
segmentor = None
if UPLOAD_MODEL or DOWNLOAD_MODEL:
  segmentor = torch.load(SEGMENTOR_MODEL_FILENAME, map_location=DEVICE).to(DEVICE)
else:
  segmentor = UNet().to(DEVICE)

In [23]:
#@title Train Models
segmentor_results = None
if TRAIN_MODEL:
  segmentor_results = train(segmentor, t_loader, v_loader, BCEWithLogitsLoss(), Adam(segmentor.parameters(), lr=INIT_LR), BinaryJaccardIndex())
  torch.save(segmentor, f'{MODELS_PATH}/{SEGMENTOR_MODEL_FILENAME}')

In [24]:
#@title Metrics
if TRAIN_MODEL:
  segmentor_results.to_csv(f'{METRICS_PATH}/metrics.csv', sep=',')

  #Segmentor Loss
  plt.plot(segmentor_results['t']['losses'], label='train loss')
  plt.plot(segmentor_results['v']['losses'], label='test loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.ylim([0, 1])
  plt.legend(loc='lower right')
  plt.title('UNet')
  plt.savefig(f'{METRICS_PATH}/unet_loss.{OUTPUT_FORMAT}')
  plt.close()

  #Segmentor Metrics
  plt.plot(segmentor_results['t']['metrics'], label='train metrics')
  plt.plot(segmentor_results['v']['metrics'], label='test metrics')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0, 1])
  plt.legend(loc='lower right')
  plt.title('UNet')
  plt.savefig(f'{METRICS_PATH}/unet_metrics.{OUTPUT_FORMAT}')
  plt.close()

##Output

In [25]:
#@title Prediction Loaders
transformations = transforms.Compose([transforms.ToPILImage(), transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()])
data = load_data() # Ground truth dataframe
dataset = SegmentationDataset(data, transformations)
row_size = int(math.sqrt(len(dataset))) # Batch of 16 since each row is 16 images long
loader = DataLoader(dataset, shuffle=False, batch_size=row_size, pin_memory=PIN_MEMORY, num_workers=os.cpu_count(), persistent_workers=True)

#Segmentor
segmentor_predictions = predict(segmentor, loader) # Prediction dataframe
segmentor_dataset = SegmentationDataset(segmentor_predictions, transformations)
segmentor_loader = DataLoader(segmentor_dataset, shuffle=False, batch_size=row_size, pin_memory=PIN_MEMORY, num_workers=os.cpu_count(), persistent_workers=True)

  0%|          | 0/16 [00:00<?, ?it/s]

In [26]:
#@title Count Tents
segmentor_predictions.to_csv(f'{PREDS_PATH}/labels.csv')

In [27]:
#@title Save Region
y_data = data.combine_first(region(loader, REGIONS_Y_PATH, 'y_regions_overview')).sort_index()
p_data = segmentor_predictions.combine_first(region(segmentor_loader, REGIONS_P_PATH, 'p_regions_overview')).sort_index()

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [28]:
#@title Save Overview

# Tiling Feature and Ground Truth Overviews
tile(loader, f'{OVERVIEW_PATH}/x_overview.{OUTPUT_FORMAT}', f'{OVERVIEW_PATH}/y_overview.{OUTPUT_FORMAT}')

# Tiling Prediction Overview
tile(segmentor_loader, None, f'{OVERVIEW_PATH}/p_overview.{OUTPUT_FORMAT}')

  0%|          | 0/16 [00:00<?, ?it/s]

Saving output/overviews/x_overview.png...
Saving output/overviews/y_overview.png...
Done.


  0%|          | 0/16 [00:00<?, ?it/s]

Saving output/overviews/p_overview.png...
Done.


In [29]:
#@title Save Overlays
print('Saving overlays...')

# Ground Truth Overlays
np.vectorize(overlay)(y_data['image_paths'], 
                      y_data['mask_paths'], 
                      np.vectorize((lambda n: f'{OVERLAY_Y_PATH}/{n}.{OUTPUT_FORMAT}'))(y_data.index), 
                      0.7)
overlay(f'{OVERVIEW_PATH}/x_overview.{OUTPUT_FORMAT}', 
        f'{OVERVIEW_PATH}/y_overview.{OUTPUT_FORMAT}', 
        f'{OVERLAY_PATH}/y_overlay.{OUTPUT_FORMAT}', 
        0.7)

# Prediction Overlays
np.vectorize(overlay)(p_data['image_paths'], 
                      p_data['mask_paths'], 
                      np.vectorize((lambda n: f'{OVERLAY_P_PATH}/{n}.{OUTPUT_FORMAT}'))(p_data.index), 
                      0.7)
overlay(f'{OVERVIEW_PATH}/x_overview.{OUTPUT_FORMAT}', 
        f'{OVERVIEW_PATH}/p_overview.{OUTPUT_FORMAT}', 
        f'{OVERLAY_PATH}/p_overlay.{OUTPUT_FORMAT}', 
        0.7)

# Ground Truth Region Overlays
np.vectorize(overlay)(np.vectorize((lambda n: f'{OVERLAY_Y_PATH}/{n}.{OUTPUT_FORMAT}'))(y_data.index),
                      y_data['region_paths'], 
                      np.vectorize((lambda n: f'{OVERLAY_YR_PATH}/{n}.{OUTPUT_FORMAT}'))(y_data.index))
overlay(f'{OVERLAY_PATH}/y_overlay.{OUTPUT_FORMAT}', 
        f'{OVERVIEW_PATH}/y_regions_overview.{OUTPUT_FORMAT}', 
        f'{OVERLAY_PATH}/y_regions_overlay.{OUTPUT_FORMAT}')

# Prediction Region Overlays
np.vectorize(overlay)(np.vectorize((lambda n: f'{OVERLAY_P_PATH}/{n}.{OUTPUT_FORMAT}'))(p_data.index), 
                      p_data['region_paths'], 
                      np.vectorize((lambda n: f'{OVERLAY_PR_PATH}/{n}.{OUTPUT_FORMAT}'))(p_data.index))
overlay(f'{OVERLAY_PATH}/p_overlay.{OUTPUT_FORMAT}', 
        f'{OVERVIEW_PATH}/p_regions_overview.{OUTPUT_FORMAT}', 
        f'{OVERLAY_PATH}/p_regions_overlay.{OUTPUT_FORMAT}')

print('Done.')

Saving overlays...
Done.


In [30]:
#@title Save Heatmaps
print('Saving heatmaps...')
heatmap(f'{OVERLAY_PATH}/y_overlay.{OUTPUT_FORMAT}', data, f'{OVERVIEW_PATH}/y_heatmap.{OUTPUT_FORMAT}', 'Actual Tents')
heatmap(f'{OVERLAY_PATH}/p_overlay.{OUTPUT_FORMAT}', segmentor_predictions, f'{OVERVIEW_PATH}/p_heatmap.{OUTPUT_FORMAT}', 'Detected Tents')
print('Done.')

Saving heatmaps...
Done.


In [ ]:
#@title Complete

# Since there's seemingly no way to reasonably wait for files.download, we have
# to wait a specified period of time before disconnecting the session.

def kill_runtime(seconds):
  with tqdm_thread(desc='Terminating session...', total=seconds, step_sec=0.5):
    time.sleep(seconds)
  print('Session terminated automatically.')
  time.sleep(1)
  runtime.unassign()

def end_runtime(seconds):
  if seconds > 0:
    print(f'Terminating Session in {seconds//60} minutes...')
    threading.Thread(target=kill_runtime, args=[seconds]).start()
  else:
    print('Session terminated automatically.')
    time.sleep(1)
    runtime.unassign()

print(f'Zipping output and finishing up.')
!7z a -tzip tent_detector_output.zip $OUTPUT_PATH

if SAVE_TO_GOOGLE_DRIVE:
  print('\nSaving to Google Drive\n')
  !rsync -arh --progress tent_detector_output.zip $G_DRIVE_STORAGE
if DOWNLOAD_OUTPUT:
  print('\nDownloading to Local Storage\n')
  files.download('tent_detector_output.zip')
if PLAY_SOUND_ON_COMPLETE:
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
print('Done.\n\n')

# Killing the runtime on complete is only useful if we save the data somewhere first
if KILL_RUNTIME_ON_COMPLETE and (SAVE_TO_GOOGLE_DRIVE or DOWNLOAD_OUTPUT):
  end_runtime(MINUTES_TO_KILL_RUNTIME if DOWNLOAD_OUTPUT else 0)

Zipping output and finishing up.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           13 folders, 1804 files, 914757114 bytes (873 MiB)

Creating archive: tent_detector_output.zip

Items to compress: 1817

  0%      0% 2 + output/overlays/p_regions_overlay.png                                                1% 2 + output/overlays/p_regions_overlay.png                                                2% 2 + output/overlays/p_regions_overlay.png                                                3% 2 + output/overl